# Getting started with Hugging Face and Amazon Sagemaker

## Binary classification on movie reviews

* https://huggingface.co/distilbert-base-uncased
* https://huggingface.co/transformers/model_doc/distilbert.html
* https://huggingface.co/datasets/imdb

# Setup

In [ ]:
!pip -q install "sagemaker>=2.31.0" "transformers>=4.4.2" "datasets[s3]>=1.5.0" --upgrade

In [ ]:
!pip -q install torch tensorflow --upgrade

In [1]:
import sagemaker

print(sagemaker.__version__)

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()

2.32.1


# Preprocessing

We are using the `datasets` library to download and preprocess the `imdb` dataset. After preprocessing, the dataset will be uploaded to our `sagemaker_session_bucket` to be used within our training job. The [imdb](http://ai.stanford.edu/~amaas/data/sentiment/) dataset consists of 25000 training and 25000 testing highly polar movie reviews.

In [2]:
from datasets import load_dataset

train_dataset, test_dataset = load_dataset('imdb', split=['train', 'test'])

print(train_dataset.shape)
print(test_dataset.shape)

Reusing dataset imdb (/home/ec2-user/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3)


(25000, 2)
(25000, 2)


In [3]:
print(train_dataset[0])

{'label': 1, 'text': 'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!'}


In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3/cache-d6a52d59d516280b.arrow
Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3/cache-10ed9b7a4517e667.arrow


In [5]:
print(train_dataset[0])

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [6]:
# Set format for pytorch
train_dataset = train_dataset.rename_column("label", "labels")
test_dataset = test_dataset.rename_column("label", "labels")

In [7]:
print(train_dataset[0])

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Upload data to S3

In [8]:
from datasets.filesystems import S3FileSystem

s3 = S3FileSystem()  

s3_prefix = 'hugging-face/demo'

training_input_path = f's3://{bucket}/{s3_prefix}/train'
train_dataset.save_to_disk(training_input_path,fs=s3)

test_input_path = f's3://{bucket}/{s3_prefix}/test'
test_dataset.save_to_disk(test_input_path,fs=s3)

In [9]:
print(training_input_path)
print(test_input_path)

s3://sagemaker-us-east-1-613904931467/hugging-face/demo/train
s3://sagemaker-us-east-1-613904931467/hugging-face/demo/test


# Fine-tuning & starting Sagemaker Training Job

In [10]:
!pygmentize ./scripts/train.py

from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import load_from_disk
import random
import logging
import sys
import argparse
import os
import torch

if __name__ == "__main__":

    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument("--epochs", type=int, default=3)
    parser.add_argument("--train-batch-size", type=int, default=32)
    parser.add_argument("--eval-batch-size", type=int, default=64)
    parser.add_argument("--warmup_steps", type=int, default=500)
    parser.add_argument("--model_name", type=str)
    parser.add_argument("--learning_rate", type=str, default=5e-5)

    # Data, model, and output directories
    parser.add_argument("--output-data-dir", type=str, default=os.environ["SM_OUTPUT_DATA_DIR"])
    parser.add_argument("--model-dir", type=s

## Fine-tune the Hugging Face model on SageMaker

In [11]:
hyperparameters={
    'epochs': 1,
    'train_batch_size': 32,
    'model_name':'distilbert-base-uncased'
}

In [12]:
import sagemaker.huggingface
from sagemaker.huggingface import HuggingFace

huggingface_estimator = HuggingFace(
    role=role,
    # Fine-tuning script
    entry_point='train.py',
    source_dir='./scripts',
    hyperparameters=hyperparameters,
    # Infrastructure
    transformers_version='4.4.2',
    pytorch_version='1.6.0',
    py_version='py36',
    instance_type='ml.p3.2xlarge',
    instance_count=1,
    # Managed Spot Training
    use_spot_instances=True,
    max_wait=3600,
    max_run=3600,
    # Disable profiling
    disable_profiler=True
)

In [13]:
huggingface_estimator.fit(
    {'train': training_input_path, 'test': test_input_path}
)

2021-04-06 11:08:28 Starting - Starting the training job...
2021-04-06 11:08:51 Starting - Launching requested ML instancesProfilerReport-1617707307: InProgress
...
2021-04-06 11:09:14 Starting - Insufficient capacity error from EC2 while launching instances, retrying!......
2021-04-06 11:10:11 Starting - Preparing the instances for training......
2021-04-06 11:11:14 Downloading - Downloading input data...
2021-04-06 11:11:52 Training - Downloading the training image..................
2021-04-06 11:15:01 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-04-06 11:15:04,498 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-04-06 11:15:04,523 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-04-06 11:15:04,533 sagemaker_pytorch_container.training INFO     Invoking use

[2021-04-06 11:15:28.658 algo-1:25 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-04-06 11:15:29.151 algo-1:25 INFO profiler_config_parser.py:102] User has disabled profiler.
[2021-04-06 11:15:29.152 algo-1:25 INFO json_config.py:91] Creating hook from json_config at /opt/ml/input/config/debughookconfig.json.
[2021-04-06 11:15:29.153 algo-1:25 INFO hook.py:199] tensorboard_dir has not been set for the hook. SMDebug will not be exporting tensorboard summaries.
[2021-04-06 11:15:29.883 algo-1:25 INFO hook.py:253] Saving to /opt/ml/output/tensors
[2021-04-06 11:15:29.883 algo-1:25 INFO state_store.py:67] The checkpoint config file /opt/ml/input/config/checkpointconfig.json does not exist.
[2021-04-06 11:15:30.663 algo-1:25 INFO hook.py:550] name:distilbert.embeddings.word_embeddings.weight count_params:23440896
[2021-04-06 11:15:30.663 algo-1:25 INFO hook.py:550] name:distilbert.embeddings.position_embeddings.weight count_params:393216
[2021-04-06 11:15:30.663 algo-1:25 INFO 

{'loss': 0.3726, 'learning_rate': 5e-05, 'epoch': 0.64}
[2021-04-06 11:19:40.103 algo-1:25 WARNING hook.py:1033] var is not Tensor or list or tuple of Tensors, module_name:distilbert.transformer BaseModelOutput
[2021-04-06 11:19:40.104 algo-1:25 WARNING hook.py:1033] var is not Tensor or list or tuple of Tensors, module_name:distilbert BaseModelOutput
[2021-04-06 11:19:40.105 algo-1:25 WARNING hook.py:1033] var is not Tensor or list or tuple of Tensors, module_name:DistilBertForSequenceClassification SequenceClassifierOutput
[2021-04-06 11:22:59.792 algo-1:25 WARNING hook.py:1033] var is not Tensor or list or tuple of Tensors, module_name:distilbert.transformer BaseModelOutput
[2021-04-06 11:22:59.793 algo-1:25 WARNING hook.py:1033] var is not Tensor or list or tuple of Tensors, module_name:distilbert BaseModelOutput
[2021-04-06 11:22:59.794 algo-1:25 WARNING hook.py:1033] var is not Tensor or list or tuple of Tensors, module_name:DistilBertForSequenceClassification SequenceClassifierO


2021-04-06 11:25:55 Uploading - Uploading generated training model
2021-04-06 11:27:57 Completed - Training job completed
Training seconds: 1003
Billable seconds: 301
Managed Spot Training savings: 70.0%


In [14]:
huggingface_estimator.sagemaker_session.logs_for_job(
    huggingface_estimator.latest_training_job.name)

2021-04-06 11:28:15 Starting - Preparing the instances for training
2021-04-06 11:28:15 Downloading - Downloading input data
2021-04-06 11:28:15 Training - Training image download completed. Training in progress.
2021-04-06 11:28:15 Uploading - Uploading generated training model
2021-04-06 11:28:15 Completed - Training job completedbash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-04-06 11:15:04,498 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-04-06 11:15:04,523 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-04-06 11:15:04,533 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-04-06 11:15:04,969 sagemaker-training-toolkit INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "test": "/opt/ml/input/data/test",
        "tra

[2021-04-06 11:15:30.673 algo-1:25 INFO hook.py:550] name:distilbert.transformer.layer.5.ffn.lin2.weight count_params:2359296
[2021-04-06 11:15:30.673 algo-1:25 INFO hook.py:550] name:distilbert.transformer.layer.5.ffn.lin2.bias count_params:768
[2021-04-06 11:15:30.673 algo-1:25 INFO hook.py:550] name:distilbert.transformer.layer.5.output_layer_norm.weight count_params:768
[2021-04-06 11:15:30.673 algo-1:25 INFO hook.py:550] name:distilbert.transformer.layer.5.output_layer_norm.bias count_params:768
[2021-04-06 11:15:30.673 algo-1:25 INFO hook.py:550] name:pre_classifier.weight count_params:589824
[2021-04-06 11:15:30.673 algo-1:25 INFO hook.py:550] name:pre_classifier.bias count_params:768
[2021-04-06 11:15:30.673 algo-1:25 INFO hook.py:550] name:classifier.weight count_params:1536
[2021-04-06 11:15:30.673 algo-1:25 INFO hook.py:550] name:classifier.bias count_params:2
[2021-04-06 11:15:30.673 algo-1:25 INFO hook.py:552] Total Trainable Params: 66955010
[2021-04-06 11:15:30.674 algo-

## Retrieve model, load it and predict

In [15]:
%%sh -s $huggingface_estimator.model_data
aws s3 cp $1 .
mkdir -p model
tar -xvzf model.tar.gz -C model

download: s3://sagemaker-us-east-1-613904931467/huggingface-pytorch-training-2021-04-06-11-08-27-719/output/model.tar.gz to ./model.tar.gz
checkpoint-500/
checkpoint-500/trainer_state.json
checkpoint-500/optimizer.pt
checkpoint-500/pytorch_model.bin
checkpoint-500/training_args.bin
checkpoint-500/scheduler.pt
checkpoint-500/config.json
pytorch_model.bin
training_args.bin
config.json


In [16]:
from transformers import AutoModel, AutoConfig, DistilBertForSequenceClassification

config = AutoConfig.from_pretrained('./model/config.json')
model = DistilBertForSequenceClassification.from_pretrained('./model/pytorch_model.bin', config=config)

print(config)
print(model)

DistilBertConfig {
  "_name_or_path": "./model/pytorch_model.bin",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.4.2",
  "vocab_size": 30522
}

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (atten

In [42]:
#inputs = tokenizer("The Phantom Menace was a really bad movie. What a waste of my life.", return_tensors='pt')
inputs = tokenizer("The Phantom Menace was an amazing movie. Jar Jar rocks!", return_tensors='pt')

print(inputs.input_ids)
print(inputs.attention_mask)

tensor([[  101,  1996, 11588, 19854,  2001,  2019,  6429,  3185,  1012, 15723,
         15723,  5749,   999,   102]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [43]:
outputs = model(**inputs)

print(outputs.logits)

tensor([[-2.2089,  2.4441]], grad_fn=<AddmmBackward>)


In [44]:
def top_class(logits):
    import torch
    import numpy as np
    softmax = torch.nn.Softmax(dim=1)
    print(softmax(logits))
    pred = np.argmax(softmax(logits).detach().numpy(), axis=1)

    return pred

In [45]:
print(top_class(outputs.logits))

tensor([[0.0094, 0.9906]], grad_fn=<SoftmaxBackward>)
[1]


In [32]:
## Fine-tune the Hugging Face model on SageMaker with Distributed Training

In [33]:
hyperparameters={
    'epochs': 16,
    'train_batch_size': 32,
    'model_name':'distilbert-base-uncased'
}

In [ ]:
huggingface_estimator = HuggingFace(
    role=role,
    # Fine-tuning script
    entry_point='train.py',
    source_dir='./scripts',
    hyperparameters=hyperparameters,
    # Infrastructure
    transformers_version='4.4.2',
    pytorch_version='1.6.0',
    py_version='py36',
    instance_type='ml.p3.16xlarge',
    instance_count=2,
    # Managed Spot Training
    use_spot_instances=True,
    max_wait=3600,
    max_run=3600,
    # Disable profiling
    disable_profiler=True
)

huggingface_estimator.fit({'train': training_input_path, 'test': test_input_path})

2021-04-06 13:14:57 Starting - Starting the training job...
2021-04-06 13:14:59 Starting - Launching requested ML instances.........
2021-04-06 13:16:49 Starting - Preparing the instances for training.........
2021-04-06 13:18:15 Downloading - Downloading input data...
2021-04-06 13:18:47 Training - Downloading the training image............
2021-04-06 13:20:57 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-04-06 13:20:56,897 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-04-06 13:20:56,976 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-04-06 13:20:58,137 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training


2021-04-06 13:21:06,147 - __main__ - INFO -  loaded train_dataset length is: 25000
2021-04-06 13:21:06,148 - __main__ - INFO -  loaded test_dataset length is: 25000
2021-04-06 13:21:06,340 - filelock - INFO - Lock 140522520212032 acquired on /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361.lock
2021-04-06 13:21:06,376 - filelock - INFO - Lock 140522520212032 released on /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361.lock
2021-04-06 13:21:06,397 - filelock - INFO - Lock 140522520212032 acquired on /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a.lock
2021-04-06 13:21:08,139 - __main__ - INFO -  loaded train_dataset length is: 25000
20

NCCL version 2.4.8+cuda11.0
NCCL version 2.4.8+cuda11.0
[2021-04-06 13:21:33.531 algo-2:26 WARNING hook.py:1033] var is not Tensor or list or tuple of Tensors, module_name:module.distilbert.transformer BaseModelOutput
[2021-04-06 13:21:33.531 algo-2:26 WARNING hook.py:1033] var is not Tensor or list or tuple of Tensors, module_name:module.distilbert BaseModelOutput
[2021-04-06 13:21:33.562 algo-2:26 WARNING hook.py:1033] var is not Tensor or list or tuple of Tensors, module_name:module SequenceClassifierOutput
[2021-04-06 13:21:34.323 algo-2:26 WARNING hook.py:1033] var is not Tensor or list or tuple of Tensors, module_name:module.distilbert.transformer BaseModelOutput
[2021-04-06 13:21:34.323 algo-2:26 WARNING hook.py:1033] var is not Tensor or list or tuple of Tensors, module_name:module.distilbert BaseModelOutput
[2021-04-06 13:21:34.328 algo-2:26 WARNING hook.py:1033] var is not Tensor or list or tuple of Tensors, module_name:module SequenceClassifierOutput
[2021-04-06 13:21:34.684

{'eval_loss': 0.27700275182724, 'eval_accuracy': 0.89432, 'eval_f1': 0.8917390591706277, 'eval_precision': 0.9140625, 'eval_recall': 0.87048, 'eval_runtime': 29.3815, 'eval_samples_per_second': 850.875, 'epoch': 1.0}
{'eval_loss': 0.2115076184272766, 'eval_accuracy': 0.91848, 'eval_f1': 0.9177894312222671, 'eval_precision': 0.9256305939788446, 'eval_recall': 0.91008, 'eval_runtime': 27.6871, 'eval_samples_per_second': 902.947, 'epoch': 2.0}
{'eval_loss': 0.2133331447839737, 'eval_accuracy': 0.917, 'eval_f1': 0.9151225099194175, 'eval_precision': 0.9363020005022181, 'eval_recall': 0.89488, 'eval_runtime': 29.0302, 'eval_samples_per_second': 861.171, 'epoch': 2.0}
{'eval_loss': 0.20018154382705688, 'eval_accuracy': 0.92448, 'eval_f1': 0.9232145762160404, 'eval_precision': 0.9389477167438782, 'eval_recall': 0.908, 'eval_runtime': 27.7887, 'eval_samples_per_second': 899.647, 'epoch': 3.0}
{'eval_loss': 0.20067191123962402, 'eval_accuracy': 0.923, 'eval_f1': 0.9212968641399895, 'eval_precis


2021-04-06 13:51:14 Uploading - Uploading generated training model
2021-04-06 13:59:53 Completed - Training job completed
Training seconds: 4996
Billable seconds: 1498
Managed Spot Training savings: 70.0%


## Fine-tune the Hugging Face model on SageMaker with Data Parallelism

In [ ]:
huggingface_estimator = HuggingFace(
    role=role,
    # Fine-tuning script
    entry_point='train.py',
    source_dir='./scripts',
    hyperparameters=hyperparameters,
    # Infrastructure
    transformers_version='4.4.2',
    pytorch_version='1.6.0',
    py_version='py36',
    instance_type='ml.p3.16xlarge',
    instance_count=2,
    # Managed Spot Training
    use_spot_instances=True,
    max_wait=3600,
    max_run=3600,
    # Disable profiling
    disable_profiler=True,
    # Data Parallelism
    distribution={'smdistributed': {'dataparallel': {'enabled': True}}}
)

huggingface_estimator.fit({'train': training_input_path, 'test': test_input_path})

2021-04-06 14:00:05 Starting - Starting the training job...
2021-04-06 14:00:20 Starting - Launching requested ML instances.........
2021-04-06 14:02:02 Starting - Preparing the instances for training.........
2021-04-06 14:03:19 Downloading - Downloading input data...
2021-04-06 14:03:50 Training - Downloading the training image...............
2021-04-06 14:06:31 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-04-06 14:06:31,092 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-04-06 14:06:31,169 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-04-06 14:06:34,222 sagemaker_pytorch_container.training INFO     Invoking SMDataParallel
2021-04-06 14:06:34,222 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-04-06 14:06:34,845 sagemak

[1,13]<stdout>:2021-04-06 14:06:47,397 - filelock - INFO - Lock 140291343550616 released on /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a.lock
[1,14]<stdout>:2021-04-06 14:06:47,416 - filelock - INFO - Lock 140359338155424 acquired on /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a.lock
[1,14]<stdout>:2021-04-06 14:06:47,416 - filelock - INFO - Lock 140359338155424 released on /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a.lock
[1,11]<stdout>:2021-04-06 14:06:47,417 - filelock - INFO - Lock 139906430984032 acquired on /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e366

[1,0]<stdout>:Running smdistributed.dataparallel v1.0.0
[1,7]<stdout>:[2021-04-06 14:06:58.678 algo-1:61 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[1,1]<stdout>:[2021-04-06 14:06:58.678 algo-1:51 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[1,2]<stdout>:[2021-04-06 14:06:58.678 algo-1:54 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[1,5]<stdout>:[2021-04-06 14:06:58.678 algo-1:59 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[1,0]<stdout>:[2021-04-06 14:06:58.678 algo-1:50 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[1,3]<stdout>:[2021-04-06 14:06:58.678 algo-1:56 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[1,10]<stdout>:[2021-04-06 14:06:58.660 algo-2:60 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[1,13]<stdout>:[2021-04-06 14:06:58.659 algo-2:66 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[1,8]<stdout>:[2021-04-06 14:06:58.659 algo-2:56 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[1,11]<stdout>:

[1,8]<stdout>:{'eval_loss': 0.6650556921958923, 'eval_accuracy': 0.75004, 'eval_f1': 0.7653486538244902, 'eval_precision': 0.7211803835538886, 'eval_recall': 0.81528, 'eval_runtime': 7.6834, 'eval_samples_per_second': 3253.784, 'epoch': 1.0}
[1,0]<stdout>:{'eval_loss': 0.6650556921958923, 'eval_accuracy': 0.75004, 'eval_f1': 0.7653486538244902, 'eval_precision': 0.7211803835538886, 'eval_recall': 0.81528, 'eval_runtime': 7.6834, 'eval_samples_per_second': 3253.783, 'epoch': 1.0}
[1,0]<stdout>:{'eval_loss': 0.265903115272522, 'eval_accuracy': 0.89548, 'eval_f1': 0.8973966309341501, 'eval_precision': 0.8812369861957277, 'eval_recall': 0.91416, 'eval_runtime': 7.2734, 'eval_samples_per_second': 3437.162, 'epoch': 2.0}
[1,8]<stdout>:{'eval_loss': 0.265903115272522, 'eval_accuracy': 0.89548, 'eval_f1': 0.8973966309341501, 'eval_precision': 0.8812369861957277, 'eval_recall': 0.91416, 'eval_runtime': 7.2789, 'eval_samples_per_second': 3434.602, 'epoch': 2.0}
[1,0]<stdout>:{'eval_loss': 0.2199

[1,0]<stdout>:{'eval_loss': 0.2790542244911194, 'eval_accuracy': 0.92256, 'eval_f1': 0.9244576244732324, 'eval_precision': 0.9023461304082876, 'eval_recall': 0.94768, 'eval_runtime': 7.3801, 'eval_samples_per_second': 3387.507, 'epoch': 11.0}
[1,8]<stdout>:{'eval_loss': 0.2790542244911194, 'eval_accuracy': 0.92256, 'eval_f1': 0.9244576244732324, 'eval_precision': 0.9023461304082876, 'eval_recall': 0.94768, 'eval_runtime': 7.3799, 'eval_samples_per_second': 3387.599, 'epoch': 11.0}
[1,8]<stdout>:{'eval_loss': 0.29104533791542053, 'eval_accuracy': 0.92412, 'eval_f1': 0.9257272620492542, 'eval_precision': 0.9065255731922398, 'eval_recall': 0.94576, 'eval_runtime': 7.388, 'eval_samples_per_second': 3383.886, 'epoch': 12.0}
[1,0]<stdout>:{'eval_loss': 0.29104533791542053, 'eval_accuracy': 0.92412, 'eval_f1': 0.9257272620492542, 'eval_precision': 0.9065255731922398, 'eval_recall': 0.94576, 'eval_runtime': 7.3846, 'eval_samples_per_second': 3385.42, 'epoch': 12.0}
[1,8]<stdout>:{'eval_loss': 


2021-04-06 14:17:16 Uploading - Uploading generated training model2021-04-06 14:17:14,101 sagemaker-training-toolkit INFO     MPI process finished.
2021-04-06 14:17:14,101 sagemaker-training-toolkit INFO     Reporting training SUCCESS

2021-04-06 14:19:31 Completed - Training job completed
Training seconds: 1944
Billable seconds: 584
Managed Spot Training savings: 70.0%
